### Входные данные

У вас имеется поток данных (генератор data_stream). Поля это случайные величины - так сделано для упрощения генерации данных. Есть три поля (названы по уровню сложности задания)

### Задание
#####Для каждого поля нужно сделать следующее:

1. Пробежаться по данным окном размера 1000 (окно сдвигается на 1, то есть следующее окно пересекается с предыдущим по 999 элементам).

2. Для каждого окна посчитайте среднее значение поля и его дисперсию. Делайте yield этих значений, получая генератор tuple. 

3. Для каждого окна найдине минимум, медиану и максимум в нём. Делайте yield этих значений, получая генератор tuple. 

Ответом, который нужно будет засабмитить в гугл форму, является среднее значение tuple по получившемуся потоку, округлённое до 3 знака после запятой. Также нужно будет прислать notebook с решением.

### Замечания

1. Обратите внимания как генерируются поля. Постарайтесь понять особенность каждого поля и как это можно использовать. Желательно, чтобы для каждого поля у вас было своё решение, максимально эффективно использующее знание об этом поле.
2. Полезные библиотеки: itertools, numpy, collections + всё что найдёте в интернете и можно поставить через pip install (указывайте имя пакета)

#### Генерация данных

In [2]:
from collections import namedtuple
import random

Record = namedtuple('Record', 'easy medium nightmare')

def data_stream():
    random_generator = random.Random(42)
    easy = 0
    for _ in xrange(10000000):
        easy += random_generator.randint(0, 2) 
        medium = random_generator.randint(0, 256 - 1)
        nightmare = random_generator.randint(0, 1000000000 - 1)
        
        yield Record(
            easy=easy,
            medium=medium,
            nightmare=nightmare
        )
        
def easy_stream():
    for record in data_stream():
        yield record.easy
        
def medium_stream():
    for record in data_stream():
        yield record.medium
        
def nightmare_stream():
    for record in data_stream():
        yield record.nightmare

#### Подсчёт среднего значения tuple по потоку

In [3]:
import numpy as np

def get_tuple_stream_mean(stream, number_of_values):
    result = np.zeros(number_of_values, dtype='object')
    count = 0. 
    for streamed_tuple in stream:
        result += streamed_tuple
        count += 1
    return [round(x, 3) for x in result / count]

#### Решение для среднего и дисперсии (общее для всех полей)

Нужно пробегаться двумя указателями и обновлять счётчики.


In [4]:
from itertools import tee

def stream_window_mean_and_variance(stream, window_size):
    value_sum = 0.
    value_sq_sum = 0.
    it_begin, it_end = tee(stream)
    for _ in xrange(window_size):
        value = next(it_end)
        value_sum += value
        value_sq_sum += value * value
        
    try:
        while True:
            yield value_sum / window_size, (value_sq_sum - value_sum * value_sum / window_size) / window_size

            next_value = next(it_end)
            prev_value = next(it_begin)
            
            value_sum += next_value
            value_sq_sum += next_value * next_value

            value_sum -= prev_value
            value_sq_sum -= prev_value * prev_value
    except StopIteration:
        pass

In [4]:
%%time
get_tuple_stream_mean(stream_window_mean_and_variance(easy_stream(), 1000), 2)

CPU times: user 2min 20s, sys: 276 ms, total: 2min 20s
Wall time: 2min 20s


[4998568.84, 83391.919]

In [5]:
%%time
get_tuple_stream_mean(stream_window_mean_and_variance(medium_stream(), 1000), 2)

CPU times: user 2min 20s, sys: 4 ms, total: 2min 20s
Wall time: 2min 20s


[127.494, 5452.516]

In [6]:
%%time
get_tuple_stream_mean(stream_window_mean_and_variance(nightmare_stream(), 1000), 2)

CPU times: user 2min 16s, sys: 8 ms, total: 2min 16s
Wall time: 2min 16s


[499909716.257, 8.322315407931899e+16]

#### Решение для минимума, медианы и максимума для поля easy

Это поле монотонно растёт в потоке. Поэтому достаточно просто хранить deque и соответствущие индексы из него. Можно поизвращаться и сделать три указателя itertool.tee (но это надо быть совсем фанатом).


In [5]:
from collections import deque
from itertools import islice

def easy_stream_min_median_max(stream, window_size):
    queue = deque()
    for value in islice(stream, window_size):
        queue.append(value)
    
    yield queue[0], queue[window_size / 2], queue[-1]
    for value in stream:
        queue.popleft()
        queue.append(value)
        yield queue[0], queue[window_size / 2], queue[-1]

In [6]:
%%time
get_tuple_stream_mean(easy_stream_min_median_max(easy_stream(), 1000), 3)

CPU times: user 2min 13s, sys: 0 ns, total: 2min 13s
Wall time: 2min 10s


[4998069.499, 4998569.34, 4999068.18]

#### Решение для минимума, медианы и максимума для поля medium

Это поле имеет очень мало значений в потоке. Поэтому можно хранить счётчики сколько каких значений мы наблюдали.

In [9]:
from itertools import tee
from numpy import cumsum
from numpy import searchsorted

def medium_stream_min_median_max(stream, window_size):
    values_counter = np.zeros(256)
    mid_value = (window_size + 1) / 2
    it_begin, it_end = tee(stream)
    for _ in xrange(window_size):
        values_counter[next(it_end)] += 1
    try:
        while True:
            cum_values = cumsum(values_counter)
            yield (
                searchsorted(cum_values, 1, side='left'),
                searchsorted(cum_values, mid_value, side='right'),
                searchsorted(cum_values, window_size, side='left')
            )

            values_counter[next(it_end)] += 1
            values_counter[next(it_begin)] -= 1
    except StopIteration:
        pass

In [10]:
%%time
get_tuple_stream_mean(medium_stream_min_median_max(medium_stream(), 1000), 3)

CPU times: user 4min 11s, sys: 4 ms, total: 4min 11s
Wall time: 4min 11s


[0.02, 127.65, 254.98]

#### Решение для минимума, медианы и максимума для поля nightmare

Это поле монотонно в общем виде. По сути надо повортить решение для easy, но надо найти дополнительную библиотеку.

In [7]:
from blist import sortedlist
from itertools import islice

def nightmare_stream_min_median_max(stream, window_size):
    window = sortedlist()
    it_begin, it_end = tee(stream)
    for _ in xrange(window_size):
        window.add(next(it_end))
    try:
        while True:
            yield window[0], window[window_size / 2], window[-1]
            window.add(next(it_end))
            window.remove(next(it_begin))
    except StopIteration:
        pass

In [8]:
%%time
get_tuple_stream_mean(nightmare_stream_min_median_max(nightmare_stream(), 1000), 3)

CPU times: user 6min 2s, sys: 0 ns, total: 6min 2s
Wall time: 6min


[1000480.564, 500313881.866, 999008045.794]

In [13]:
%%time
get_tuple_stream_mean(nightmare_stream_min_median_max(medium_stream(), 1000), 3)

CPU times: user 5min 42s, sys: 0 ns, total: 5min 42s
Wall time: 5min 42s


[0.02, 127.65, 254.98]